# We Rate Dogs Data Analysis 

We are going to analyze data coming from the WeRateDogs twitter channel. This project aims to practice thorough data wrangling techniques. Additionally, the goal is to find out interesting facts and write a report.

Your goal: wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. 

## Table of Contents

1. [Introduction](#introduction)
2. [Data Wrangling](#data-wrangling)
    1. [Gathering data](#gathering-data)
    2. [Assessing data](#assessing-data)
    3. [Cleaning data](#cleaning-data) 
3. [Analysis and Visualization](#analysis-and-visualization)
4. [Reporting](#reporting)

## Introduction <a name="introduction"></a>
Some introduction text, formatted in heading 2 style

To get started, let's import our libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import json
import requests
import os.path
import re
%matplotlib inline

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 5000)
pd.set_option('display.max_colwidth', -1)

## Data Wrangling <a name="data-wrangling"></a>
The first paragraph text

### Gathering Data <a name="gathering-data"></a>
Read in the first data set: WeRateDogs Twitter archive provided by Udacity.

In [2]:
# read in twitter archive 
twitter_dogs_archive = pd.read_csv('twitter-archive-enhanced-2.csv')
twitter_dogs_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


Download and read in image predictions file provided by Udacity.

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# download file programmatically
response = requests.get(url)
    
# create new file if not existent
if not os.path.exists('image-predictions.tsv'):
    file = open('image-predictions.tsv', 'w')
    file.close()

# open file and write file content
with open('image-predictions.tsv', 'wb') as file_image_predictions:
        file_image_predictions.write(response.content)
        

In [3]:
# load image predictions into data frame
image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
image_predictions.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


### Twitter API request using tweepy 

In [ ]:
# hide login details
with open('logins.json') as login_file:
    logins = json.load(login_file)

def get_secret(setting, logins=logins):
    """Get login setting or fail with ImproperlyConfigured"""
    try:
        return logins[setting]
    except KeyError:
        raise ImproperlyConfigured("Set the {} setting.".format(setting))

In [ ]:
# retrieve Twitter login details 
consumer_key = get_secret('consumer_key')
consumer_secret = get_secret('consumer_secret')
access_token = get_secret('access_token')
access_secret = get_secret('access_secret')

In [ ]:
# access Twitter API
import tweepy

# Redirect to Twitter to authorize
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# Get access token
auth.set_access_token(access_token, access_secret)

# api instance
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [ ]:
# get tweets from WeRateDogs Twitter timeline 
start = time.time()
error_ids = []
print("Start requesting WeRateDogs tweets.")
with open('tweet_json.txt', 'w', encoding='utf-8') as file:
    file.write("[\n")
    for index, tweet_id in enumerate(twitter_dogs_archive.tweet_id.values):
        ranking = index + 1 
        try:
            # Twitter API request using specific tweet_id
            status = api.get_status(tweet_id, tweet_mode='extended')
            status_json = json.dumps(status._json)

            # write json object
            file.write(status_json)
            if ranking < len(twitter_dogs_archive.tweet_id.values):
                file.write(",")
            file.write("\n")
            
            # This cell is slow so print ranking to gauge time remaining
            print(ranking, '-', tweet_id)

        except tweepy.TweepError as e:
            # catch erroneos
            error_ids.append(tweet_id)
            e = e.response.text
            print(e)
    file.write("]")
end = time.time()
print("Process finisheed. Time elapsed: ", round((end-start) / 60, 2), "min." )

In [4]:
tweets = []
with open('tweet_json.txt', 'r') as file:
    data = json.loads(file.read())
    for i in range(0, len(data)):
        record = {"id": data[i]["id"], "retweet_count": data[i]['retweet_count'], "favorite_count": data[i]["favorite_count"]}
        tweets.append(record)

tweets_df = pd.DataFrame(tweets)
tweets_df.head()


,favorite_count,id,retweet_count
0,37683,892420643555336193,8215
1,32373,892177421306343426,6076
2,24378,891815181378084864,4017
3,41004,891689557279858688,8370
4,39208,891327558926688256,9075


In [ ]:
# save erroneous ids
errors_df= pd.DataFrame(error_ids)
errors_df.to_csv('errors.csv',index=False)

In [ ]:
# load errors
errors_df = pd.read_csv('errors.csv')
errors_df.info()

 query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count.

### Assassing Data <a name="assessing-data"></a>
The paragraph text

#### Visual assessment

In [ ]:
twitter_dogs_archive

In [ ]:
tweets_df

In [ ]:
image_predictions

#### Programmatic assessment

In [ ]:
# Assess twitter dogs enhanced.
twitter_dogs_archive.info()

In [ ]:
# show all retweets
retweets = twitter_dogs_archive[twitter_dogs_archive.retweeted_status_id.notna()]['retweeted_status_id'].values.astype(np.int64)
twitter_dogs_archive[twitter_dogs_archive.retweeted_status_id.notna()]

In [ ]:
# check if original tweets are in twitter archive
retweets_reduced = retweets
for retweet in retweets:
    if retweet in twitter_dogs_archive.tweet_id.values:
        index = np.argwhere(retweets_reduced == retweet)
        retweets_reduced = np.delete(retweets_reduced, index)
print(len(twitter_dogs_archive[twitter_dogs_archive.retweeted_status_id.notna()]) - len(retweets_reduced), "entries out of the retweets are contained in the twitter archive.\n")

In [111]:
# Show all replies to assess if they are relevant for our research
replies = twitter_dogs_archive[twitter_dogs_archive.in_reply_to_status_id.notna()]['tweet_id'].values.astype(np.int64)
twitter_dogs_archive[twitter_dogs_archive.in_reply_to_status_id.notna()][['tweet_id','text']]

valid_replies = [863079547188785154, 856526610513747968, 847617282490613760, 802265048156610565, 786051337297522688, 
                 766714921925144576, 704871453724954624,675870721063669760, 675707330206547968, 669353438988365824]

,tweet_id,text
30,886267009285017600,@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution
55,881633300179243008,@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s
64,879674319642796034,@RealKentMurphy 14/10 confirmed
113,870726314365509632,@ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs
148,863427515083354112,"@Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy content above I'm leaning towards 12/10"
149,863079547188785154,"Ladies and gentlemen... I found Pipsy. He may have changed his name to Pablo, but he never changed his love for the sea. Pupgraded to 14/10 https://t.co/lVU5GyNFen"
179,857214891891077121,@Marc_IRL pixelated af 12/10
184,856526610513747968,"THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY HI AFTER ALL. PUPGRADED TO A 14/10. WOULD BE AN HONOR TO FLY WITH https://t.co/p1hBHCmWnA"
186,856288084350160898,@xianmcguire @Jenna_Marbles Kardashians wouldn't be famous if as a society we didn't place enormous value on what they do. The dogs are very deserving of their 14/10
188,855862651834028034,@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research


In [122]:
dogs_clean.query('tweet_id == 669353438988365824')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,favorite_count,retweet_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
2166,669353438988365824,6.678065e+17,4.196984e+09,2015-11-25 03:14:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Tessa. She is also very pleased after finally meeting her biological father. 10/10 https://t.co/qDS1aCqppv,https://twitter.com/dog_rates/status/669353438988365824/photo/1,10,10,Tessa,None,649,266,https://pbs.twimg.com/media/CUoFZTyW4AE70iD.jpg,1.0,teddy,0.379656,False,Pembroke,0.212343,True,chow,0.096995,True


In [59]:
# assess names
twitter_dogs_archive.name.value_counts()

None              745
a                  55
Charlie            12
Lucy               11
Oliver             11
Cooper             11
Lola               10
Penny              10
Tucker             10
Bo                  9
Winston             9
Sadie               8
the                 8
Bailey              7
Toby                7
an                  7
Buddy               7
Daisy               7
Stanley             6
Milo                6
Leo                 6
Jax                 6
Dave                6
Koda                6
Rusty               6
Jack                6
Scout               6
Bella               6
Oscar               6
very                5
Sammy               5
Bentley             5
Alfie               5
Larry               5
Finn                5
Gus                 5
Sunny               5
George              5
Phil                5
Oakley              5
Chester             5
Louis               5
Duke                4
Jerry               4
Clarence            4
Riley     

In [5]:
# after finding typical mistakes, I'm checking if there is a pattern to recover names  
determiners = ["a", "an", "the", "officially", "old", "just", "quite", "getting", "actually", "mad", "not", "by", "very", "one", "this", "life", "all", "None"]

# loop trough names column and print each text of the text column whenever name equals determiner
for i, row in twitter_dogs_archive.iterrows():
    if row['name'] in determiners:
        print(i , "-", row['text'])

5 - Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh
7 - When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq
12 - Here's a puppo that seems to be on the fence about something haha no but seriously someone help her. 13/10 https://t.co/BxvuXk0UCm
24 - You may not have known you needed to see this today. 13/10 please enjoy (IG: emmylouroo) https://t.co/WZqNqygEyV
25 - This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp
30 - @NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution
32 - RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo
35 - I have a new hero and his name is Howard. 14/10 https://t.co/gzLHboL7Sk
37 - Here we have a corgi und

939 - So this just changed my life. 13/10 please enjoy  https://t.co/dsv4xAtfv7
943 - RT @dog_rates: HEY PUP WHAT'S THE PART OF THE HUMAN BODY THAT CONNECTS THE FOOT AND THE LEG? 11/10 so smart https://t.co/XQ1tRUmO3z
944 - Nothing better than a doggo and a sunset. 10/10 majestic af https://t.co/xVSodF19PS
945 - Hooman used Pokeball
*wiggle*
*wiggle*
Doggo broke free 
10/10 https://t.co/bWSgqnwSHr
946 - Here are three doggos completely misjudging an airborne stick. Decent efforts tho. All 9/10 https://t.co/HCXQL4fGVZ
947 - Hopefully this puppo on a swing will help get you through your Monday. 11/10 would push https://t.co/G54yClasz2
948 - Here's a doggo trying to catch some fish. 8/10 futile af (vid by @KellyBauerx) https://t.co/jwd0j6oWLE
949 - RT @dog_rates: Everyone needs to watch this. 13/10 https://t.co/Bb3xnpsWBC
956 - Please stop sending it pictures that don't even have a doggo or pupper in them. Churlish af. 5/10 neat couch tho https://t.co/u2c9c7qSg8
958 - Here's a pupper that

1542 - Here we see 33 dogs posing for a picture. All get 11/10 for superb cooperation https://t.co/TRAri5iHzd
1543 - Downright majestic af 12/10 https://t.co/WFh2FEbYzj
1545 - Someone sent me this without any context and every aspect of it is spectacular. 13/10 please enjoy https://t.co/Rxrd4hPmp4
1550 - We normally don't rate birds but I feel bad cos this one forgot to fly south for the winter. 9/10 just wants a bath https://t.co/o47yitCn9N
1552 - This pupper just wants to say hello. 11/10 would knock down fence for https://t.co/A8X8fwS78x
1554 - When you have a ton of work to do but then remember you have tomorrow off. 10/10 https://t.co/MfEaMUFYTx
1557 - When you stumble but recover quickly cause your crush is watching. 12/10 https://t.co/PMeq6IedU7
1560 - This pupper is sprouting a flower out of her head. 12/10 revolutionary af https://t.co/glmvQBRjv4
1564 - Please send dogs. I'm tired of seeing other stuff like this dangerous pirate. We only rate dogs. Thank you... 10/10 https://t

2327 - This is a southern Vesuvius bumblegruff. Can drive a truck (wow). Made friends with 5 other nifty dogs (neat). 7/10 https://t.co/LopTBkKa8h
2328 - Oh goodness. A super rare northeast Qdoba kangaroo mix. Massive feet. No pouch (disappointing). Seems alert. 9/10 https://t.co/Dc7b0E8qFE
2329 - Those are sunglasses and a jean jacket. 11/10 dog cool af https://t.co/uHXrPkUEyl
2330 - Unique dog here. Very small. Lives in container of Frosted Flakes (?). Short legs. Must be rare 6/10 would still pet https://t.co/XMD9CwjEnM
2331 - Here we have a mixed Asiago from the Galápagos Islands. Only one ear working. Big fan of marijuana carpet. 8/10 https://t.co/tltQ5w9aUO
2332 - Look at this jokester thinking seat belt laws don't apply to him. Great tongue tho 10/10 https://t.co/VFKG1vxGjB
2333 - This is an extremely rare horned Parthenon. Not amused. Wears shoes. Overall very nice. 9/10 would pet aggressively https://t.co/QpRjllzWAL
2334 - This is a funny dog. Weird toes. Won't come down. Love

In [127]:
# Assess all observations with None values to doublecheck if names were missed
twitter_dogs_archive[twitter_dogs_archive.name == "None"][['tweet_id','text']]


,tweet_id,text
5,891087950875897856,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh
7,890729181411237888,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq
12,889665388333682689,Here's a puppo that seems to be on the fence about something haha no but seriously someone help her. 13/10 https://t.co/BxvuXk0UCm
24,887343217045368832,You may not have known you needed to see this today. 13/10 please enjoy (IG: emmylouroo) https://t.co/WZqNqygEyV
25,887101392804085760,This... is a Jubilant Antarctic House Bear. We only rate dogs. Please only send dogs. Thank you... 12/10 would suffocate in floof https://t.co/4Ad1jzJSdp
30,886267009285017600,@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution
32,886054160059072513,RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo
35,885518971528720385,I have a new hero and his name is Howard. 14/10 https://t.co/gzLHboL7Sk
37,885167619883638784,Here we have a corgi undercover as a malamute. Pawbably doing important investigative work. Zero control over tongue happenings. 13/10 https://t.co/44ItaMubBf
41,884441805382717440,"I present to you, Pup in Hat. Pup in Hat is great for all occasions. Extremely versatile. Compact as h*ck. 14/10 (IG: itselizabethgales) https://t.co/vvBOcC2VdC"


In [39]:
# Assess whether names were extracted correctly.  
for i,row in twitter_dogs_archive.iterrows():
    if not row['name'] in row['text'] and (row['name'] != "None"):
        print(row.tweet_id, row.text, row.name, "\n")

In [46]:
# Assess if denominators are all 10. Multiple dogs in one picture/videos
for i,row in twitter_dogs_archive.iterrows():
    if not row['rating_denominator'] == 10:
        print(row.tweet_id, "\n", row.text)
        print("Numerator: ", row.rating_numerator, "\nDenominator: ", row.rating_denominator, "\n")


#
irrelevant_tweets = [835246439529840640, 832088576586297345, 810984652412424192, 686035780142297088, 684225744407494656, 682808988178739200  ]
multiple_dogs = [758467244762497024, 709198395643068416  ]

835246439529840640 
 @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho
Numerator:  960 
Denominator:  0 

832088576586297345 
 @docmisterio account started on 11/15/15
Numerator:  11 
Denominator:  15 

820690176645140481 
 The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd
Numerator:  84 
Denominator:  70 

810984652412424192 
 Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. 
Keep Sam smiling by clicking and sharing this link:
https://t.co/98tB8y7y7t https://t.co/LouL5vdvxx
Numerator:  24 
Denominator:  7 

775096608509886464 
 RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…
Numerator:  9 
Denominator:  11 

758467244762497024 
 Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE
Numerator:  165 
Denominator:  150 

740373189193256964 
 After so many r

In [52]:
dogs_clean.query('tweet_id == 709198395643068416') 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,favorite_count,retweet_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1417,709198395643068416,NaN,NaN,2016-03-14 02:04:08 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",https://twitter.com/dog_rates/status/709198395643068416/photo/1,45,50,NaN,None,2511,671,https://pbs.twimg.com/media/CdeUKpcWoAAJAWJ.jpg,1.0,borzoi,0.490783,True,wire-haired_fox_terrier,0.083513,True,English_setter,0.083184,True


In [ ]:
twitter_dogs_archive[twitter_dogs_archive.tweet_id == 679062614270468096]

In [ ]:
twitter_dogs_archive.doggo.value_counts()

In [ ]:
twitter_dogs_archive.floofer.value_counts()

In [ ]:
twitter_dogs_archive.pupper.value_counts()

In [ ]:
twitter_dogs_archive.puppo.value_counts()

In [ ]:
# assess texts
twitter_dogs_archive.text

In [ ]:
print("Number of duplicated tweet ids:", len(twitter_dogs_archive[twitter_dogs_archive.tweet_id.duplicated(keep=False)]))
twitter_dogs_archive[twitter_dogs_archive.tweet_id.duplicated()]

In [ ]:
# Tweets containing "We only rate dogs caught my attention", however it seems to be a joke for dogs that don't look like dogs. 
# Print every row that contains We only rate dogs" or "We. Only. Rate. Dogs."
import re
pattern = re.compile(r'we.? only.? rate.? dogs', re.IGNORECASE)
for i, row in dogs_clean.iterrows():
    if pattern.search(row['text']):
        print(row['text'])

##### Assessing tweets data

In [ ]:
# looking at errors
print("Number of non-existing tweet ids: ", len(errors_df))

In [ ]:
tweets_df[tweets_df['retweet_count'].isnull()]

##### Assessing image predictions data

In [ ]:
image_predictions.info()

In [ ]:
image_predictions.tweet_id.duplicated().sum()

In [ ]:
image_predictions.iloc[2059]['jpg_url']

In [ ]:
# check if retweets are in image predictions -> no retweets in image predictions
for retweet in retweets_reduced:
    if retweet in image_predictions.tweet_id.values:
        print(retweet)

**Quality**  

_**twitter archive table**_
- irrelevant retweets
- irrelevant columns in_reply_to_status_id, in_reply_to_user_id
- Incorrect dog names, determiners like: a, an, the, officially, old, just, quite, getting, actually, mad, not, by, very, one, this, life, all, None
- ratings are not normalized
- missing dog names
- incorrect null values in dog stages. None should be NaN.
- Erroneous data types (timestamp, source, dog_stage)
- Source value is wrapped in anchor tag
- Text contains mentions of users, e.g. @NonWhiteHat
- Contains tweets that are replies to other treets --> Remove if in_reply_to_status_id/in_reply_to_user_id is not NaN
- Some tweets contain a link using t.co, twitter's url shortener. They are not active anymore. Working url is included in expanded_urls
- Row 47, 59, 62,91 not a valid observation (We only rate dogs-comments)
- Incorrectly-extracted or None as names, e.g. a row 56, None should be NaN
- Incorrect demoninators (not 10)
- Incomparable rating numerators.
- Tweets with missing photos
- Incorrect dog stages
- dog statuses should be NaN values instead of a string of None

- dogs with multiple dog stage in the following tweet_ids: 854010172552949760, 817777686764523521, 808106460588765185, 802265048156610565, 801115127852503040, 785639753186217984,781308096455073793, 775898661951791106, 770093767776997377, 759793422261743616, 751583847268179968, 741067306818797568, 733109485275860992, 855851453814013952

_**image predictions table**_
- dog races in p1, p2, p3 contain underscores, some are uppercase, some are lowercase
- dog races contain non dog- races, e.g. hen, cock, personal_computer, shopping_cart, box-turtle... --> p_dog is False in this case
- contains 281 fewer entries compared to twitter archive table

_**tweets table**_ 
- contains 17 fewer ids compared to twitter archive due to errors during twitter request 

**Tidiness**  

_**twitter archive table**_
- doggo, floofer, pupper, puppo should be one column
- 3 separate tables of the same purpose
- Multiple urls in expanded_urls (some are duplicates)


### Cleaning Data <a name="cleaning-data"></a>
#### Tidiness


In [6]:
# Create copies of data frames
dogs_clean = twitter_dogs_archive
tweets_clean = tweets_df
images_clean = image_predictions

**dogs: _doggo, floofer, pupper, puppo should be one column_**

_**Define**_

Create a dog_stage column to assign the status of dog to each tweet. Use the 4 separate columns of doggo floofer, pupper and puppo. Leave the value empty if all of the 4 columns contain "None".

_**Code**_

In [7]:
# create a dog status column by using doggo column
column_names = ['tweet_id','in_reply_to_status_id','in_reply_to_user_id', \
                'timestamp','source','text','retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp',\
                'expanded_urls','rating_numerator','rating_denominator','name']

# add column for none dog type
dogs_clean['None'] = "placeholder"

for i, row in dogs_clean.iterrows():
    if row.loc['doggo'] == row.loc['floofer'] == row.loc['pupper'] == row.loc['puppo'] == 'None':
        dogs_clean.at[i, 'None'] = "None"

# melt dog stages into rows
dogs_clean = pd.melt(dogs_clean, id_vars=column_names, var_name='placeholder', value_name='dog_stage')

# remove all excess rows and columns
for i, row in dogs_clean.iterrows():
    if not row.loc['placeholder'] == row.loc['dog_stage']:
        dogs_clean = dogs_clean.drop([i])
        
dogs_clean = dogs_clean.drop(['placeholder'], axis=1).reset_index(drop=True)


_**Test**_

In [8]:
# Data frame must have a min of 2356 non-null values plus 14 tweets with multiple values for dog stages
dogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2370 entries, 0 to 2369
Data columns (total 14 columns):
tweet_id                      2370 non-null int64
in_reply_to_status_id         79 non-null float64
in_reply_to_user_id           79 non-null float64
timestamp                     2370 non-null object
source                        2370 non-null object
text                          2370 non-null object
retweeted_status_id           183 non-null float64
retweeted_status_user_id      183 non-null float64
retweeted_status_timestamp    183 non-null object
expanded_urls                 2311 non-null object
rating_numerator              2370 non-null int64
rating_denominator            2370 non-null int64
name                          2370 non-null object
dog_stage                     2370 non-null object
dtypes: float64(4), int64(3), object(7)
memory usage: 259.3+ KB


In [9]:
# no changes in value counts of dog stages
dogs_clean.dog_stage.value_counts()

None       1976
pupper     257 
doggo      97  
puppo      30  
floofer    10  
Name: dog_stage, dtype: int64

_**3 separate tables of the same purpose**_

_**Define**_

Join dogs table and tweets table using 'tweet_id', respectively 'id', removing non-matching tweet_ids. Than, join new dogs table and image predictions table on their common tweet_id. Keep all entries with non-matching dog race predictions to not loose entries. 

_**Code**_

In [10]:
# merge dogs and tweets table 
dogs_clean = pd.merge(dogs_clean, tweets_clean, how='inner',  left_on='tweet_id', right_on='id').drop(['id'], axis=1)

# merge dogs and image predications tables 
dogs_clean = pd.merge(dogs_clean, images_clean, how='left',  on='tweet_id')

_**Test**_

In [11]:
# should have all 2353 entries after dropping the missing tweets
# all columns are combined in one table 
dogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2353 entries, 0 to 2352
Data columns (total 27 columns):
tweet_id                      2353 non-null int64
in_reply_to_status_id         79 non-null float64
in_reply_to_user_id           79 non-null float64
timestamp                     2353 non-null object
source                        2353 non-null object
text                          2353 non-null object
retweeted_status_id           169 non-null float64
retweeted_status_user_id      169 non-null float64
retweeted_status_timestamp    169 non-null object
expanded_urls                 2294 non-null object
rating_numerator              2353 non-null int64
rating_denominator            2353 non-null int64
name                          2353 non-null object
dog_stage                     2353 non-null object
favorite_count                2353 non-null int64
retweet_count                 2353 non-null int64
jpg_url                       2079 non-null object
img_num                       2079

_**Quality**_

_**irrelevant retweets**_

_**Define**_  
Keep only rows containing a null value in 'retweeted_status_id'. Subsequentially, remove irrelevant columns: 'retweeted_status_id', 'retweeted_status_user_id' and 'retweeted_status_timestamp'.

_**Code**_

In [12]:
dogs_clean = dogs_clean[dogs_clean.retweeted_status_id.isna()]
dogs_clean.drop(['retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp'], axis=1, inplace=True)

_**Test**_

In [13]:
# After removing 169 retweet observations, we should have 2184 observations left.  
dogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2184 entries, 0 to 2352
Data columns (total 24 columns):
tweet_id                 2184 non-null int64
in_reply_to_status_id    79 non-null float64
in_reply_to_user_id      79 non-null float64
timestamp                2184 non-null object
source                   2184 non-null object
text                     2184 non-null object
expanded_urls            2126 non-null object
rating_numerator         2184 non-null int64
rating_denominator       2184 non-null int64
name                     2184 non-null object
dog_stage                2184 non-null object
favorite_count           2184 non-null int64
retweet_count            2184 non-null int64
jpg_url                  2002 non-null object
img_num                  2002 non-null float64
p1                       2002 non-null object
p1_conf                  2002 non-null float64
p1_dog                   2002 non-null object
p2                       2002 non-null object
p2_conf                 

_**Incorrect dog names, determiners like: a, an, the, officially, old, just, quite, getting, actually, mad, not, by, very, one, this, life, all, None**_

_**Define**_  
Scan through all missing names, which are all names represented by a, an, the, officially, old, just, quite, getting, actually, mad, not, by, very, one, this, life, all or None. Identify dog names in the text column of the structure "named (something)". Replace the name if there is a name pattern match in the text field. If there is no matching pattern, fill the cell by a null value. 

_**Code**_

In [20]:
pattern1 = re.compile(r'named [A-Za-z]+')
pattern2 = re.compile(r'name\.? is\.? [A-Za-z]+', re.IGNORECASE)

def extract_name(row):
    name_result1 = re.search(pattern1, row['text'])
    name_result2 = re.search(pattern2, row['text'])
    
    if name_result1:
        new_name = name_result1.group().split()[1]
    elif name_result2:
        new_name = name_result2.group().split()[2]
    else:
        new_name = np.nan
    return new_name
    
# loop trough names column and print each text of the text column whenever name equals determiner
for i, row in dogs_clean.iterrows():
    if row['name'] in determiners:
        new_name = extract_name(row)
        dogs_clean.at[i, 'name'] = new_name
        
        
        

_**Test**_

In [21]:
dogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2184 entries, 0 to 2352
Data columns (total 24 columns):
tweet_id                 2184 non-null int64
in_reply_to_status_id    79 non-null float64
in_reply_to_user_id      79 non-null float64
timestamp                2184 non-null object
source                   2184 non-null object
text                     2184 non-null object
expanded_urls            2126 non-null object
rating_numerator         2184 non-null int64
rating_denominator       2184 non-null int64
name                     1459 non-null object
dog_stage                2184 non-null object
favorite_count           2184 non-null int64
retweet_count            2184 non-null int64
jpg_url                  2002 non-null object
img_num                  2002 non-null float64
p1                       2002 non-null object
p1_conf                  2002 non-null float64
p1_dog                   2002 non-null object
p2                       2002 non-null object
p2_conf                 

In [22]:
# all determiners are gone, names were correctly extracted
dogs_clean.name.value_counts()


Lucy              11
Cooper            11
Oliver            11
Charlie           10
Tucker            9 
Penny             9 
Lola              8 
Sadie             8 
Winston           8 
Toby              8 
Daisy             7 
Bella             6 
Oscar             6 
Jack              6 
Jax               6 
Koda              6 
Bo                6 
Stanley           6 
Bailey            6 
Scout             5 
Buddy             5 
Louis             5 
Milo              5 
Bentley           5 
Dave              5 
Rusty             5 
Chester           5 
Maggie            5 
Leo               5 
Zoey              4 
Oakley            4 
Alfie             4 
Duke              4 
Phil              4 
Sophie            4 
George            4 
Brody             4 
Boomer            4 
Archie            4 
Bear              4 
Winnie            4 
Loki              4 
Finn              4 
Chip              4 
Reese             4 
Max               4 
Cassie            4 
Jeffrey      

_**Missing dog names**_

_**Define**_  
Replace null value in dog names by the dog's that was extracted from the tweet's text.   

_**Code**_

In [23]:
missing_names = [(826204788643753985, "Dew"), (854120357044912130, "Cooper"), (778039087836069888, "Max"), 
                   (685547936038666240, "Jack"),(878604707211726852, "Martha"), (863079547188785154, "Pipsy"), 
                   (856526610513747968, "Charly"), (847617282490613760, "Cannon"), (844979544864018432, "Toby"),
                   (836001077879255040, "Atlas"), (831650051525054464, "Blue"), (811647686436880384, "Augie"), 
                   (778408200802557953, "Loki"), (758041019896193024, "Teagan"), (740373189193256964, "Bretagne"), 
                   (708026248782585858, "Frank"), (704871453724954624, "Pipsie"), (695064344191721472, "Charles"), 
                   (692142790915014657, "Teddy"), (685681090388975616, "Jack"), (685325112850124800, "Tristan"), 
                   (684538444857667585, "Pippa"), (678023323247357953, "Reese"), (677687604918272002, "Cindy"),
                   (676590572941893632, "Bubbles"), (675870721063669760, "Yoshi"), (669684865554620416, "Dug"), 
                   (668142349051129856, "Oliver")]


In [24]:
# loop missing names and match corresponding tweet with name provided in the set
for el in missing_names:
    # find row with matching tweet_id
    for i, row in dogs_clean.iterrows():
        if row['tweet_id'] == el[0]:
            dogs_clean.at[i, 'name'] = el[1]


_**Test**_

In [26]:
# should have 1431+28= 1459 names 
dogs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2184 entries, 0 to 2352
Data columns (total 24 columns):
tweet_id                 2184 non-null int64
in_reply_to_status_id    79 non-null float64
in_reply_to_user_id      79 non-null float64
timestamp                2184 non-null object
source                   2184 non-null object
text                     2184 non-null object
expanded_urls            2126 non-null object
rating_numerator         2184 non-null int64
rating_denominator       2184 non-null int64
name                     1459 non-null object
dog_stage                2184 non-null object
favorite_count           2184 non-null int64
retweet_count            2184 non-null int64
jpg_url                  2002 non-null object
img_num                  2002 non-null float64
p1                       2002 non-null object
p1_conf                  2002 non-null float64
p1_dog                   2002 non-null object
p2                       2002 non-null object
p2_conf                 

In [27]:
# test if new names are correct
dogs_clean.name.value_counts()

Lucy              11
Cooper            11
Oliver            11
Charlie           10
Tucker            9 
Penny             9 
Lola              8 
Sadie             8 
Winston           8 
Toby              8 
Daisy             7 
Bella             6 
Oscar             6 
Jack              6 
Jax               6 
Koda              6 
Bo                6 
Stanley           6 
Bailey            6 
Scout             5 
Buddy             5 
Louis             5 
Milo              5 
Bentley           5 
Dave              5 
Rusty             5 
Chester           5 
Maggie            5 
Leo               5 
Zoey              4 
Oakley            4 
Alfie             4 
Duke              4 
Phil              4 
Sophie            4 
George            4 
Brody             4 
Boomer            4 
Archie            4 
Bear              4 
Winnie            4 
Loki              4 
Finn              4 
Chip              4 
Reese             4 
Max               4 
Cassie            4 
Jeffrey      

In [102]:
# assess dogs_clean
for el in dogs_clean[dogs_clean.name.isna()][['text', 'tweet_id']].values:
    print(el)
    
# irrelevant_tweets = [840696689258311684, 840698636975636481, 835246439529840640, 674606911342424069]

# two_in one = [('Burke', 'Dexter',808106460588765185), (Cletus, Jerome, Alejandro, Burp, & Titson,709198395643068416),
(689599056876867584 = 33 dogs), (669037058363662336, "Pancho","Peaches"), (668221241640230912, "Bo", "Smittens"), (666835007768551424, "Cupit and Prencer" ")]

["Here's a very large dog. He has a date later. Politely asked this water person to check if his breath is bad. 12/10 good to go doggo https://t.co/EMYIdoblMR"
 872967104147763200]
['Never doubt a doggo 14/10 https://t.co/AbBLh2FZCH' 871102520638267392]
["I have stumbled puppon a doggo painting party. They're looking to be the next Pupcasso or Puppollock. All 13/10 would put it on the fridge https://t.co/cUeDMlHJbq"
 858843525470990336]
["Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel"
 855851453814013952]
["Here's a puppo participating in the #ScienceMarch. Cleverly disguising her own doggo agenda. 13/10 would keep the planet habitable for https://t.co/cMhq16isel"
 855851453814013952]
["At first I thought this was a shy doggo, but it's actually a Rare Canadian Floofer Owl. Amateurs would confuse the two. 11/10 only send dogs https://t.co/TXdT3tmuYk"
 854010172552949760]
["At

In [55]:
# names were replaced by a name in their respective text column
# for i, row in clean.iterrows():
#     if row['name'] in determiners:
#         new_name = extract_name(row)
#         clean.at[i, 'name'] = new_name

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,favorite_count,retweet_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,31067,7134,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,1.0,Pembroke,0.511319,True,Cardigan,0.451038,True,Chihuahua,0.029248,True
1,884162670584377345,NaN,NaN,2017-07-09 21:29:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Yogi. He doesn't have any important dog m...,https://twitter.com/dog_rates/status/884162670...,12,10,Yogi,doggo,19828,2887,https://pbs.twimg.com/media/DEUtQbzW0AUTv_o.jpg,1.0,German_shepherd,0.707046,True,malinois,0.199396,True,Norwegian_elkhound,0.049148,True
2,872967104147763200,NaN,NaN,2017-06-09 00:02:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a very large dog. He has a date later. ...,https://twitter.com/dog_rates/status/872967104...,12,10,None,doggo,26726,5288,https://pbs.twimg.com/media/DB1m871XUAAw5vZ.jpg,2.0,Labrador_retriever,0.476913,True,Chesapeake_Bay_retriever,0.174145,True,German_short-haired_pointer,0.092861,True
3,871515927908634625,NaN,NaN,2017-06-04 23:56:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Napolean. He's a Raggedy East Nicaragu...,https://twitter.com/dog_rates/status/871515927...,12,10,Napolean,doggo,19794,3395,https://pbs.twimg.com/media/DBg_HT9WAAEeIMM.jpg,2.0,komondor,0.974781,True,briard,0.020041,True,swab,0.003228,False
4,871102520638267392,NaN,NaN,2017-06-03 20:33:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never doubt a doggo 14/10 https://t.co/AbBLh2FZCH,https://twitter.com/animalcog/status/871075758...,14,10,None,doggo,20536,5412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


_**Incorrect null values in dog stages. None should be NaN.**_

_**Define**_  
Replace every None string into a numpy null value in name and dog_stage columns. 

_**Code**_

In [ ]:
dogs_clean['dog_stage'].replace("None", np.nan, inplace=True)

_**Test**_

In [ ]:
# dog stages should have 2184-1828=356 entries, since there were 1828 Nones  
# names should have 2184-87=1497 entries, since there were 687 Nones 
dogs_clean.info()


In [49]:
dogs_clean.query('tweet_id == 722974582966214656') 

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,favorite_count,retweet_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1320,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20,NaN,None,4232,1646,https://pbs.twimg.com/media/CgiFjIpWgAA4wVp.jpg,1.0,Great_Dane,0.246762,True,Greater_Swiss_Mountain_dog,0.126131,True,Weimaraner,0.085297,True


_**Ratings are not nomalized**_

_**Define**_   
First, correct wrongly extracted ratings. Than, find all ratings that don't have a 10 denominator and remove the corresponding row. 

_**Code**_

In [69]:
# falsy ratings and their corrected and normalized numerator
corrected_ratings = [{'tweet_id': 820690176645140481, 'rating_numerator': 12}, 
                     {'tweet_id': 722974582966214656, 'rating_numerator': 13},
                     {'tweet_id': 716439118184652801, 'rating_numerator': 11},
                     {'tweet_id': 713900603437621249, 'rating_numerator': 11},
                     {'tweet_id': 710658690886586372, 'rating_numerator': 10},
                     {'tweet_id': 704054845121142784, 'rating_numerator': 12},
                     {'tweet_id': 697463031882764288, 'rating_numerator': 11},
                     {'tweet_id': 684222868335505415, 'rating_numerator': 11},
                     {'tweet_id': 682962037429899265, 'rating_numerator': 6},
                     {'tweet_id': 677716515794329600, 'rating_numerator': 12},
                     {'tweet_id': 675853064436391936, 'rating_numerator': 11},
                     {'tweet_id': 666287406224695296, 'rating_numerator': 11},
                    ]

In [104]:
# replace falsy numerators by their corrected and normalized values. Set their denominator to 10.
for rating in corrected_ratings:
    dogs_clean.loc[dogs_clean['tweet_id'] == rating['tweet_id'], 'rating_numerator'] = rating['rating_numerator']
    dogs_clean.loc[dogs_clean['tweet_id'] == rating['tweet_id'], 'rating_denominator'] = 10

In [105]:
# remove all entries containing a denominator other than 10
dogs_clean = dogs_clean[dogs_clean.rating_denominator == 10]

_**Test**_

In [106]:
# Test if ratings were replaced correctly
for rating in corrected_ratings:
    print(dogs_clean.loc[clean['tweet_id'] == rating['tweet_id']][['rating_numerator', 'rating_denominator']])

     rating_numerator  rating_denominator
747  12                10                
Empty DataFrame
Columns: [rating_numerator, rating_denominator]
Index: []
      rating_numerator  rating_denominator
1320  13                10                
      rating_numerator  rating_denominator
1350  11                10                
      rating_numerator  rating_denominator
1375  11                10                
      rating_numerator  rating_denominator
1398  10                10                
      rating_numerator  rating_denominator
1483  12                10                
      rating_numerator  rating_denominator
1551  11                10                
      rating_numerator  rating_denominator
1708  11                10                
      rating_numerator  rating_denominator
1727  6                 10                
      rating_numerator  rating_denominator
1821  12                10                
      rating_numerator  rating_denominator
1873  11                1

In [107]:
# all rating_denominators are 10
dogs_clean[dogs_clean.rating_denominator != 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_stage,favorite_count,retweet_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


_**Test**_

_**Irrelevant columns 'in reply to status id', 'in reply to user id'**_

_**Define**_  
Drop 'in reply to status id' and 'in reply to user id' columns in data set.

_**Code**_

In [ ]:
dogs_clean.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace=True)

_**Test**_

In [ ]:
# 'in reply to status id' and 'in reply to user id' columns were removed
dogs_clean.info()

_**Erroneous data types (timestamp, source, dog stage)**_

_**Define**_  

Convert timestamp into datetime format. Convert source and dog stage into categorical data. 

_**Code**_

In [ ]:
# To datetime
dogs_clean.timestamp = pd.to_datetime(dogs_clean.timestamp)

# To category
dogs_clean.source = dogs_clean.source.astype('category')
dogs_clean.dog_stage = dogs_clean.dog_stage.astype('category')

_**Test**_

In [ ]:
dogs_clean.info()

## Analysis and Visualization <a name="analysis-and-visualization"></a>
The paragraph text

- Most popular dog names/breed vs. retweets, favortes, breed
- most popular dog content 
- rating statistics
- popularity of the account - over time
- Where are users from?
- most popular hashtags
- what race is associated with which dogtype

## Reporting <a name="reporting"></a>
The paragraph text